In [11]:
#Завантаження бібліотек

import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout, Embedding, Conv1D, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [12]:
# Завантаження та попередня обробка відгуків

new_reviews = pd.read_csv("harry_potter_reviews.csv")
new_reviews['sentiment'] = (new_reviews['rating'] > 3).astype(int)
fan_comment = new_reviews['comment'].tolist()
sentiments = new_reviews['sentiment'].tolist()

new_reviews.head()

,user_id,user_sex,user_age,user_country,rating,comment,favourite_character,date,sentiment
0,0,female,50,Germany,2.5,"""The transitions between scenes were awkward, ...",Severus Snape,2004-12-27,0
1,1,female,23,Spain,4.0,"""Severus Snape's role adds an intriguing layer.""",Severus Snape,2003-11-22,1
2,2,male,32,France,3.0,"""The pacing was a bit slow, but the characters...",Ron Weasley,2005-09-16,0
3,3,female,24,Turkey,4.5,"""Hagrid's love for magical creatures is heartw...",Rubeus Hagrid,2002-09-17,1
4,4,female,40,Spain,5.0,"""Neville Longbottom's courage is awe-inspiring.""",Neville Longbottom,2004-10-17,1


In [13]:
# Токенізація і доповнення
ourmaxWordsCount = 1000

new_tokenizer = Tokenizer(num_words=ourmaxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
new_tokenizer.fit_on_texts(fan_comment)
data = new_tokenizer.texts_to_sequences(fan_comment)
max_text_len = 60

data_pad = pad_sequences(data, maxlen=max_text_len)

print(data_pad)

[[  0   0   0 ...  14 195  41]
 [  0   0   0 ... 153   2 109]
 [  0   0   0 ...   7   4  80]
 ...
 [  0   0   0 ...  52   1 222]
 [  0   0   0 ... 104   4  80]
 [  0   0   0 ... 146 106 182]]


In [14]:
# Підготовка набору даних

X = np.array(data_pad)
Y = np.array([[1, 0] if sentiment == 1 else [0, 1] for sentiment in sentiments])
print(X, Y)

[[  0   0   0 ...  14 195  41]
 [  0   0   0 ... 153   2 109]
 [  0   0   0 ...   7   4  80]
 ...
 [  0   0   0 ...  52   1 222]
 [  0   0   0 ... 104   4  80]
 [  0   0   0 ... 146 106 182]] [[0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0

In [15]:
from tensorflow.keras.layers import GRU

# Будування моделі
our_model = Sequential()
our_model.add(Embedding(ourmaxWordsCount, 128))
our_model.add(Dropout(0.25))
our_model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
our_model.add(GRU(128, return_sequences=True))
our_model.add(Dropout(0.5))
our_model.add(GRU(64, return_sequences=False))
our_model.add(Dropout(0.5))
our_model.add(Dense(64, activation='relu'))
our_model.add(Dropout(0.5))
our_model.add(Dense(2, activation='softmax'))

our_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

# Тренування моделі
history = our_model.fit(X, Y, batch_size=32, epochs=40, validation_split=0.2, callbacks=[lr_reduction])


Epoch 1/40
13/13 [==============================] - 7s 193ms/step - loss: 0.6442 - accuracy: 0.7245 - val_loss: 0.5656 - val_accuracy: 0.7374 - lr: 0.0010
Epoch 2/40
13/13 [==============================] - 2s 119ms/step - loss: 0.5446 - accuracy: 0.7679 - val_loss: 0.5387 - val_accuracy: 0.7374 - lr: 0.0010
Epoch 3/40
13/13 [==============================] - 2s 180ms/step - loss: 0.4702 - accuracy: 0.7653 - val_loss: 0.4640 - val_accuracy: 0.7374 - lr: 0.0010
Epoch 4/40
13/13 [==============================] - 2s 119ms/step - loss: 0.2853 - accuracy: 0.8724 - val_loss: 0.4171 - val_accuracy: 0.7677 - lr: 0.0010
Epoch 5/40
13/13 [==============================] - 2s 127ms/step - loss: 0.1388 - accuracy: 0.9541 - val_loss: 0.4752 - val_accuracy: 0.8384 - lr: 0.0010
Epoch 6/40
13/13 [==============================] - 1s 109ms/step - loss: 0.0908 - accuracy: 0.9821 - val_loss: 0.4461 - val_accuracy: 0.8889 - lr: 0.0010
Epoch 7/40
13/13 [==============================] - ETA: 0s - loss: 0.

In [16]:
import random

# Функція передбачення
def predict_random_review(our_model, new_tokenizer, reviews, max_text_len):
    random_review = random.choice(reviews)

    print(f"Відгук: {random_review}")
    sequence = new_tokenizer.texts_to_sequences([random_review])
    padded_sequence = pad_sequences(sequence, maxlen=max_text_len)
    prediction = our_model.predict(padded_sequence)
    sentiment = "Позитивний" if np.argmax(prediction) == 0 else "Негативний"

    print(f"Передбачений настрій відгуку: {sentiment}")

# Приклад використання функції
predict_random_review(our_model, new_tokenizer, fan_comment, max_text_len)

Відгук: "The magical creatures showcased in the film are visually stunning."
1/1 [==============================] - 1s 1s/step
Передбачений настрій відгуку: Позитивний


Висновок: Створення рекурентної нейронної мережі (RNN) з використанням моделі GRU (Gated Recurrent Unit) для семантичного аналізу тексту дозволяє ефективно розпізнавати та класифікувати текстові дані з точністю. Модель GRU є покращенням стандартної RNN, яка допомагає уникнути проблеми з втратою градієнту під час тренування і покращує здатність моделі до довгострокової залежності.